In [1]:
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import glob,os
from tqdm import tqdm
import datetime
import proplot as plot
from netCDF4 import Dataset
from wrf import getvar
%matplotlib inline

plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

In [2]:
import json,pickle
def save_to_pickle(loc=None,var=None):
    with open(loc,"wb") as f:
        pickle.dump(var,f)
    return None

#save_to_pickle(loc='../output/ctrl_thetaall',var=ctrl_theta)
#save_to_pickle(loc='../output/ncrf36_thetaall',var=ncrf36_theta)
#save_to_pickle(loc='../output/ncrf60_thetaall',var=ncrf60_theta)

path = '/work2/08350/tg876493/stampede2/python_codes/tcg_radiation/output/'
file = [path+'ctrl_thetaall',path+'ncrf36_thetaall',path+'ncrf60_thetaall']

output = []
for i in range(len(file)):
    with open(file[i],'rb') as f:
        output.append(pickle.load(f))

theta_dict = {'CTRL_theta':output[0],'NCRF36_theta':output[1],'NCRF60_theta':output[2]}

In [3]:
origin_path = '/scratch/06040/tg853394/tc/output/redux/maria/'
#Function####################################################
def read_some_azimuth_fields(fileloc=None,fieldname=None):
    dict_name = {}
    for inx,obj in enumerate(fileloc):
        field_read = xr.open_dataset(obj)
        dict_name[fieldname[inx]] = field_read
    return dict_name
#############################################################
ctrlvar_dict = read_some_azimuth_fields(fileloc=[origin_path+'ctl/azim_T_0-144hr.nc',
                                                origin_path+'ctl/azim_H_DIABATIC_0-144hr.nc'],
                                        fieldname=['T','HDIA'])
ncrf60var_dict = read_some_azimuth_fields(fileloc=[origin_path+'ncrf_60h/azim_T_60-144hr.nc',
                                                  origin_path+'ncrf_60h/azim_H_DIABATIC_60-144hr.nc'],
                                          fieldname=['T','HDIA'])
ncrf36var_dict = read_some_azimuth_fields(fileloc=[origin_path+'ncrf_36h/azim_T_36-144hr.nc',
                                                  origin_path+'ncrf_36h/azim_H_DIABATIC_36-144hr.nc'],
                                          fieldname=['T','HDIA'])

In [4]:
def nearest_index(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx.values
# To be consistent with azimuthal files
########################################################################################################
ctrl_thetaA,ncrf36_thetaA,ncrf60_thetaA = np.swapaxes(np.asarray(theta_dict['CTRL_theta']),0,1),\
np.swapaxes(np.asarray(theta_dict['NCRF36_theta']),0,1),np.swapaxes(np.asarray(theta_dict['NCRF60_theta']),0,1)

# Heating Rate
#########################################################################################################
r0,r200=(nearest_index(ctrlvar_dict['T']['T'].radius,0),nearest_index(ctrlvar_dict['T']['T'].radius,200))
r300,r500=(nearest_index(ctrlvar_dict['T']['T'].radius,300),nearest_index(ctrlvar_dict['T']['T'].radius,500))
ctrl_thetagrad = np.gradient(ctrl_thetaA,60*60,axis=0)
ncrf36_thetagrad = np.gradient(ncrf36_thetaA,60*60,axis=0)
ncrf60_thetagrad = np.gradient(ncrf60_thetaA,60*60,axis=0)

In [65]:
ctrl_t2,ctrl_t4 = nearest_index(ctrlvar_dict['T']['T'].time/24,2),nearest_index(ctrlvar_dict['T']['T'].time/24,4.5)

### PCA

In [6]:
def normalize_inner(var=None,outerradius=None):
    PWper_ctrl = []
    for indx in range(len(var[:,0,0])):
        temp = var[indx,:,:outerradius]-np.nanmean(var[indx,:,:outerradius],axis=(0,1))
        PWper_ctrl.append((temp-np.nanmean(temp))/np.nanstd(temp))
    del temp
    return np.asarray(PWper_ctrl)

output_ctrl,output_ncrf36,output_ncrf60 = [],[],[]
for presindex in range(len(ctrl_thetagrad[0,:,0,0])):
    output_ctrl.append(normalize_inner(ctrl_thetagrad[:,presindex,:,:],r500))
for presindex in range(len(ncrf36_thetagrad[0,:,0,0])):
    output_ncrf36.append(normalize_inner(ncrf36_thetagrad[:,presindex,:,:],r500))
for presindex in range(len(ncrf60_thetagrad[0,:,0,0])):
    output_ncrf60.append(normalize_inner(ncrf60_thetagrad[:,presindex,:,:],r500))

##### Do we need to smooth the data?

In [23]:
from scipy.ndimage import gaussian_filter
output_ctrl = []
for presindex in range(len(ctrl_thetagrad[0,:,0,0])):
    output_ctrl.append(gaussian_filter(normalize_inner(ctrl_thetagrad[:,presindex,:,:],r500),0.9))

In [24]:
output_ctrl = np.swapaxes(np.asarray(output_ctrl),0,1)

##### flatten matrices

In [70]:
ctrl_thetagradf = np.asarray([output_ctrl[i,:,:,:r500].flatten() \
                              for i in range(len(output_ctrl[ctrl_t2:ctrl_t4,0,0,0]))])
#ncrf36_thetagradf = np.asarray([ncrf36_thetagrad[i,:,:,:].flatten() for i in range(len(ncrf36_thetagrad[:,0,0,0]))])
#ncrf60_thetagradf = np.asarray([ncrf60_thetagrad[i,:,:,:].flatten() for i in range(len(ncrf60_thetagrad[:,0,0,0]))])

In [71]:
ctrl_thetagradf.shape

(60, 2344680)

In [72]:
from sklearn.decomposition import PCA
import time
start_time = time.time()
skpca = PCA()
skpca.fit(ctrl_thetagradf.copy())
print("--- %s seconds ---" % (time.time() - start_time))

--- 82.86938643455505 seconds ---


In [60]:
# Day2-Day4,r200-r500
skpca.explained_variance_ratio_.cumsum()

array([0.15355587, 0.23026858, 0.29874358, 0.36216944, 0.4171096 ,
       0.46792486, 0.5170024 , 0.5608981 , 0.60340166, 0.64370966,
       0.6808311 , 0.71616274, 0.7474792 , 0.77834386, 0.8061763 ,
       0.8329123 , 0.85652566, 0.87757015, 0.8975714 , 0.91466135,
       0.9286815 , 0.9416799 , 0.9519864 , 0.9609796 , 0.96921825,
       0.97555274, 0.9814083 , 0.9861134 , 0.9898217 , 0.9925368 ,
       0.99461806, 0.99615514, 0.9973289 , 0.9982197 , 0.99884224,
       0.99921864, 0.99948496, 0.9996757 , 0.9997976 , 0.9998785 ,
       0.9999256 , 0.9999641 , 0.99998146, 0.99999046, 0.9999965 ,
       0.9999987 , 0.9999999 , 0.9999999 ], dtype=float32)

In [69]:
# Day2-Day4.5,r200-r500
skpca.explained_variance_ratio_.cumsum()

array([0.12709036, 0.19826677, 0.2554981 , 0.31013787, 0.3577393 ,
       0.40351126, 0.44621742, 0.48730913, 0.52680194, 0.5620174 ,
       0.59554017, 0.62876236, 0.65848607, 0.6880091 , 0.71542555,
       0.74203694, 0.767515  , 0.79093295, 0.81217635, 0.8332258 ,
       0.8525363 , 0.8702053 , 0.8865003 , 0.90127486, 0.91515374,
       0.92704886, 0.9376402 , 0.9470263 , 0.95511913, 0.96235967,
       0.96895885, 0.9745965 , 0.9790088 , 0.98283327, 0.98627657,
       0.9893389 , 0.99162287, 0.99350125, 0.99492776, 0.996151  ,
       0.9971056 , 0.9978784 , 0.9984785 , 0.9989111 , 0.99920595,
       0.9994284 , 0.9996045 , 0.99972594, 0.9998158 , 0.9998781 ,
       0.9999186 , 0.999952  , 0.9999718 , 0.9999832 , 0.9999904 ,
       0.9999955 , 0.99999774, 0.99999946, 1.        , 1.        ],
      dtype=float32)

In [73]:
# Day2-Day4.5,r0-r500
skpca.explained_variance_ratio_.cumsum()

array([0.10066482, 0.17190647, 0.22670533, 0.27941045, 0.32491195,
       0.36937666, 0.4127213 , 0.45403045, 0.4917999 , 0.52766585,
       0.5630492 , 0.597558  , 0.62982416, 0.66192144, 0.6929607 ,
       0.722309  , 0.7486962 , 0.7738347 , 0.79748595, 0.8186267 ,
       0.83864397, 0.85681283, 0.8745965 , 0.8918057 , 0.9069503 ,
       0.9207477 , 0.93258786, 0.94256544, 0.95114964, 0.95910686,
       0.965903  , 0.97184527, 0.9769644 , 0.9810928 , 0.9845227 ,
       0.98775834, 0.9904834 , 0.99257916, 0.9943377 , 0.99574757,
       0.99679244, 0.99763256, 0.9982626 , 0.9987617 , 0.9991294 ,
       0.99938095, 0.9995577 , 0.99969894, 0.9997946 , 0.9998647 ,
       0.9999122 , 0.99994135, 0.9999666 , 0.999982  , 0.9999897 ,
       0.999995  , 0.99999803, 0.99999946, 1.0000001 , 1.0000001 ],
      dtype=float32)